In [3]:
input_file = "/kaggle/input/txtfile/requirements.txt"  # Replace with your file name
output_file = "/kaggle/working/install_requirements.txt"

with open(input_file, "r") as infile, open(output_file, "w") as outfile:
    for line in infile:
        line = line.strip()
        if line:  # Avoid processing empty lines
            outfile.write(f"pip install {line}\n")

print(f"Modified file saved as {output_file}")


Modified file saved as /kaggle/working/install_requirements.txt


In [4]:
!pip install einops

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.7 MB/s eta 0:00:00


In [5]:
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.3/390.3 kB 8.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.0/345.0 kB 5.5 MB/s eta 0:00:00:00:01


In [6]:
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

In [7]:
import torch
import torchvision.transforms as T
from PIL import Image
from transformers import AutoModel, AutoTokenizer

# Image preprocessing constants
IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD = (0.229, 0.224, 0.225)

def build_transform(input_size):
    MEAN, STD = IMAGENET_MEAN, IMAGENET_STD
    transform = T.Compose([
        T.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
        T.Resize((input_size, input_size), interpolation=T.InterpolationMode.BICUBIC),
        T.ToTensor(),
        T.Normalize(mean=MEAN, std=STD)
    ])
    return transform

def dynamic_preprocess(image, image_size=448, use_thumbnail=False):
    orig_width, orig_height = image.size
    aspect_ratio = orig_width / orig_height

    # calculate the existing image aspect ratio and target ratios
    target_ratios = [(i, j) for n in range(1, 13) for i in range(1, n + 1) for j in range(1, n + 1) if i * j <= n]
    target_ratios = sorted(target_ratios, key=lambda x: x[0] * x[1])

    # find the closest aspect ratio to the target
    best_ratio = (1, 1)  # Replace with actual logic to find closest ratio
    target_width = image_size * best_ratio[0]
    target_height = image_size * best_ratio[1]

    # resize the image
    resized_img = image.resize((target_width, target_height))
    processed_images = [resized_img]  # For single image, we just return one image
    if use_thumbnail:
        thumbnail_img = image.resize((image_size, image_size))
        processed_images.append(thumbnail_img)

    return processed_images

def load_image(image_file, input_size=448):
    image = Image.open(image_file).convert('RGB')
    transform = build_transform(input_size=input_size)
    images = dynamic_preprocess(image, image_size=input_size, use_thumbnail=True)
    pixel_values = [transform(img) for img in images]
    pixel_values = torch.stack(pixel_values)
    return pixel_values

In [8]:
import google.generativeai as genai
from PIL import Image
import base64
from openai import OpenAI
from transformers import PaliGemmaForConditionalGeneration, PaliGemmaProcessor
import torchvision.transforms as T
from transformers import AutoModel, AutoTokenizer
from transformers import AutoProcessor, LlavaForConditionalGeneration



# Configure the API key
GOOGLE_API_KEY = "API KEY"
genai.configure(api_key=GOOGLE_API_KEY)

def call_gemini_1_5_pro(text_query, image_path, temperature=0.4):
    """Call Gemini 1.5 Pro Vision model with text and image input."""
    model = genai.GenerativeModel('gemini-1.5-pro')

    # Open the image
    img = Image.open(image_path)

    try:
        # Generate content
        response = model.generate_content(
            [text_query, img],
            generation_config=genai.types.GenerationConfig(
                candidate_count=1,
                max_output_tokens=1024,
                temperature=temperature
            )
        )
        # Return the generated response
        return response.candidates[0].content.parts[0].text
    except Exception as e:
        print(f"Error during Gemini 1.5 Pro Vision call: {e}")
        return ""

def call_gemini_1_5_flash(text_query, image_path, temperature=0.4):
    """Call Gemini 1.5 Flash Vision model with text and image input."""
    model = genai.GenerativeModel('gemini-1.5-flash')

    # Open the image
    img = Image.open(image_path)

    try:
        # Generate content
        response = model.generate_content(
            [text_query, img],
            generation_config=genai.types.GenerationConfig(
                candidate_count=1,
                max_output_tokens=1024,
                temperature=temperature
            )
        )
        # Return the generated response
        return response.candidates[0].content.parts[0].text
    except Exception as e:
        print(f"Error during Gemini 1.5 Flash Vision call: {e}")
        return ""


def call_gpt_vision(text_query, image_path,temperature=0.4):
    
    # Initialize the client with API key
    openai.api_key = os.environ.get("OPENAI_API_KEY")
    
    # Get the base64-encoded image
    base64_image = encode_image(image_path)
    
    # Construct the message with both the text query and image
    messages = [
        {
            "role": "user",
            "content": f"{text_query}\nBelow is the image I want you to analyze:",
        },
        {
            "role": "user",
            "content": f"![image](data:image/jpeg;base64,{base64_image})",
        },
    ]
    
    # Send the chat completion request
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=messages,
        max_tokens=300
    )
    
    return response['choices'][0]['message']['content']

In [9]:
def call_internvl2_vision(text_query, image_path):
    # Model and Tokenizer initialization
    path = 'OpenGVLab/InternVL2_5-1B'
    model = AutoModel.from_pretrained(
        path,
        torch_dtype=torch.bfloat16,
        low_cpu_mem_usage=True,
        use_flash_attn=True,
        trust_remote_code=True
    ).eval().cuda()

    tokenizer = AutoTokenizer.from_pretrained(path, trust_remote_code=True, use_fast=False)

    # Load and preprocess the image
    pixel_values = load_image(image_path, input_size=448).to(torch.bfloat16).cuda()

    # Set generation configuration
    generation_config = dict(max_new_tokens=512, do_sample=True)

    # Create question based on text_query
    question = f'<image>\n{text_query}\n'

    # Perform inference
    response = model.chat(tokenizer, pixel_values, question, generation_config)
    
    # Return the model's response
    return response

In [10]:
def call_llava_vision(text_query, image_path, temperature=0.4):
    
    # Model ID
    model_id = "xtuner/llava-phi-3-mini-hf"

    # Load Model
    model = LlavaForConditionalGeneration.from_pretrained(
        model_id, 
        torch_dtype=torch.float16, 
        low_cpu_mem_usage=True, 
    ).to(0)

    # Load Processor
    processor = AutoProcessor.from_pretrained(model_id)

    print(image_path)

    # Load Image from Local File
    raw_image = Image.open(image_path)  # Open the local image

    # Prepare Inputs for the Model
    prompt = f"<|user|>\n<image>\n{text_query}<|end|>\n<|assistant|>\n"
    inputs = processor(prompt, raw_image, return_tensors='pt').to(0, torch.float16)

    # Set generation configuration
    generation_config = dict(max_new_tokens=150, do_sample=True)
    # Generate Output
    output = model.generate(**inputs, max_new_tokens=generation_config['max_new_tokens'], 
                            do_sample=generation_config['do_sample'])

    # Decode and Return the Response
    response = processor.decode(output[0][2:], skip_special_tokens=True)
    return response

In [11]:
pip install sentence-transformers


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 6.4 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.


In [12]:
import os
import torch
import numpy as np
import pandas as pd
import json
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util

# Function definitions remain the same
def call_vision(text_query, image_path, model):
    if model == "gemini_1":
        response = call_gemini_1_5_flash(text_query, image_path)
    elif model == "gemini_2":
        response = call_gemini_1_5_pro(text_query, image_path)
    elif model == "gpt4":
        respone = call_gpt_vision(text_query, image_path)
    elif model == "intern_vl2":
        response =  call_internvl2_vision(text_query, image_path)
    elif model == "llava":
        response =  call_llava_vision(text_query, image_path)
    else:
        raise NotImplementedError
    return response


In [13]:
def prepare_vision_prompt(dataset):
    prompts = []
    for row in dataset:
        # Extract required details from each dataset row
        id = row["id"]
        img_path_list = row["image_path"]
        img_path = img_path_list[0]
        conversations = row["conversations"]

        # Extract the question and true response from conversations
        prompt = next((conv["value"] for conv in conversations if conv["from"] == "human"), None)
        true_response = next((conv["value"] for conv in conversations if conv["from"] == "gpt"), None)

        # Construct and append the prompt dictionary
        prompts.append({
            "id": id,
            "text_query": prompt,
            "image_path": f"/kaggle/input/{img_path}",
            "true_label": true_response
        })
    return prompts


In [14]:
from sentence_transformers import SentenceTransformer, util

def check_correctness(vision_response, true_label, similarity_threshold=0.9):
    # Load the model
    model = SentenceTransformer('all-MiniLM-L6-v2')

    # Generate embeddings
    embedding1 = model.encode(vision_response, convert_to_tensor=True)
    embedding2 = model.encode(true_label, convert_to_tensor=True)

    # Compute cosine similarity
    similarity_score = util.pytorch_cos_sim(embedding1, embedding2).item()
    
    print(f"Semantic Similarity: {similarity_score:.2f}")

    return similarity_score


In [15]:
# Model and options setup (replace argparse with direct variables)
model = "gemini_2"  # Specify the model to use

result_dir = "Pub_med_benchmark"

if not os.path.exists(result_dir):
    os.makedirs(result_dir)

In [16]:
!pip install datasets

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [17]:
# Load the dataset\

from datasets import load_dataset

dataset = load_dataset("FreedomIntelligence/PubMedVision", "PubMedVision_Alignment_VQA")["train"]

# Map the dataset to restructure the data
dataset = dataset.map(lambda x: {
    "id": x["id"].split('/')[-1],
    "image_path": x["image"],
    "conversations": x["conversations"]
})

# Select a subset for processing
dataset = dataset.select([12,13,18,20,23])


# Prepare prompts
prompts = prepare_vision_prompt(dataset)

# Initialize results DataFrame
results = []

for prompt_data in tqdm(prompts, desc="Evaluating Prompts"):
    # Extract prompt details
    question_id = prompt_data["id"]
    prompt = prompt_data["text_query"]
    image_path = prompt_data["image_path"]
    true_label = prompt_data["true_label"]
    # Call the vision model
    vision_response = (
        call_vision(prompt, image_path, model)
        if model in ['gemini_1', 'gemini_2', 'gpt4','intern_vl2','llava']
        else ""
    )
    
    print(f"True Label: {true_label}\nVision Response: {vision_response}")
 
    # Evaluate correctness
    correct = check_correctness(vision_response, true_label)

    # Append results
    results.append({
        "id": question_id,
        "prompt" : prompt,
        "response": {"vision": vision_response},
        "semantic_similarity": {"vision": correct},
        "true_label": true_label
    })

# Convert results to a DataFrame
df_results = pd.DataFrame(results)

# Save results to a JSON file
result_path = f"{result_dir}/{model}_PubMED_results.json"
df_results.to_json(result_path, orient="records", indent=4)


README.md:   0%|          | 0.00/2.38k [00:00<?, ?B/s]

PubMedVision_Alignment_VQA.json:   0%|          | 0.00/710M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/646759 [00:00<?, ? examples/s]

Map:   0%|          | 0/646759 [00:00<?, ? examples/s]

Evaluating Prompts:   0%|          | 0/5 [00:00<?, ?it/s]

...

Semantic Similarity: 0.01
Error during Gemini 1.5 Pro Vision call: 429 Resource has been exhausted (e.g. check quota).
True Label: The provided image appears to be a histological section of skeletal muscle tissue. The tissue displays numerous necrotic and regenerating muscle fibers, as indicated by the hematoxylin-eosin (H&E) staining. The image also shows an upregulation of major histocompatibility complex class I (MHC-I) expression, as well as sarcoplasmic deposition of the membrane attack complex (MAC) on the muscle fibers. These findings are characteristic of an immune-mediated necrotizing myopathy (IMNM), a condition associated with autoimmune disorders. The image provides representative examples of the histological features observed in both seropositive and seronegative forms of IMNM.
Vision Response: 


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Evaluating Prompts: 100%|██████████| 5/5 [00:35<00:00,  7.12s/it]

Semantic Similarity: 0.00
